# MIT-GSL Uruguay 

## January 2020

-----

# Week - 2 | Lesson - 03 
# NLP: Product embeddings

1. After introducing the concept of embeddings through NLP, we extend the notion of embeddings to other settings
2. Note that embeddings are dense continuous representations for discrete, sparse tokens - this makes embeddings widely applicable
3. We will use the concept of embeddings to understand the world of e-commerce better 

---

# Word2Vec and its Applications to Market-Basket Data

# Instacart Grocery Dataset

### Source: https://www.instacart.com/datasets/grocery-shopping-2017

1. Instacart is an online grocery delivery service
2. They have made available 3M grocery orders for over 200K users
3. They provide between 4 to 100 orders for each user and each order contains the sequence of products purchased
4. We also have a brief description of the products

### Overview:
1. We will use this data to build an understanding of word embeddings and investigate their application to downstream tasks
2. For this purpose, we will consider each purchase basket to be a sentence with an unordered sequence of words

---

# 0. Import Modules

In [2]:
# ==============================================
# 0. Module imports
# ==============================================

import pandas as pd
pd.options.display.max_colwidth = 100
import numpy as np
from itertools import product
import csv

# w2v
import gensim

# text processing
from nltk import sent_tokenize, word_tokenize
from nltk.tokenize import ToktokTokenizer
import string
import re # regular expressions
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics.pairwise import cosine_similarity

# T-Sne
#import umap
#from openTSNE import TSNE, TSNEEmbedding, affinity, initialization
#from openTSNE import initialization
#from openTSNE.callbacks import ErrorLogger
from sklearn.manifold import TSNE


# utils
from sklearn import utils


# parallel processing
import multiprocessing
from joblib import delayed, Parallel

# time code
import time

# 2-d visualiztion
%matplotlib inline
from ggplot import *
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

import os
#os.chdir("/pool001/madhavk/gsl-uruguay/W-02-NLP/")

In [4]:
# =========================================================
# set directories
# =========================================================

import os
EC2 = True  # If using EC2 (for data directory/paths)

# Select path based off of local or remote
if not EC2:
    wd = "/pool001/madhavk/gsl-uruguay/W-02-NLP/"
else:
    wd = "/home/ubuntu/machine_learning_aws/"
os.chdir(wd)

EC2 = True  # If using EC2 (for data directory/paths)
if not EC2:
    # raw data
    raw_data_dir = "nlp-data/in-grocery/instacart_2017_05_01/"
    # processed data
    process_dir = "nlp-data/in-grocery/prepared-data/"  
else:
    # raw data
    raw_data_dir = "data/in-grocery/instacart_2017_05_01"
    # processed data
    process_dir = "data/in-grocery/prepared-data/"

In [5]:
# =========================================================
# processed files
# =========================================================

os.listdir(process_dir)

['orders-split-v1.csv',
 'all-orders-wide-v1.csv',
 'all-orders-long-v1.csv',
 'products-merged-v1.csv']

In [6]:
# =========================================================
# global parameters
# =========================================================

# show entire value of cell in pandas
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 500)

# number of cpus
cpus = multiprocessing.cpu_count()
f"Number of CPUs: {cpus}"

'Number of CPUs: 4'

------

# 1. Import grocery data

## 1.1. Order level data

In [7]:
# =========================================================
# order-level data
# =========================================================

orders_wide = pd.read_csv(process_dir + "all-orders-wide-v1.csv")
# This data set has one row per order with the products ordered in the product_id column. 
# Products are separated by space.
print(orders_wide.shape)

(3152555, 4)


In [8]:
display(orders_wide.head(10))

,order_id,product_id,num_products,eval
0,2,33120 28985 9327 45918 30035 17794 40141 1819,8,prior
1,3,33754 24838 17704 21903 17668 46667 17461 32665,8,prior
2,4,46842 39758 27761 10054 21351 22598 34862 40285 17616 25146 32645 41276,12,prior
3,5,13176 47329 27966 23909 48370 13245 27360 6348 40878 6184 48002 20914 37011 12962 45698 41176 48366 47209 46522 38693 48825 8479,22,prior
4,7,34050 46802,2,prior
5,9,21405 47890 11182 2014 29193 34203 14992 31506 23288 44533 18362 432 3990 14183,14,prior
6,10,24852 4796 31717 47766 4605 1529 21137 22122 34134 27156 14992 49235 26842 3464 25720,15,prior
7,11,30162 5994 1313 31506,4,prior
8,12,30597 15221 43772 37886 37215 34335 26910 38888 38050 29471,10,prior
9,13,17330 27407 35419 196 44635 26878 25783 41290 33198 23020 36086 3800 25952,13,prior


In [9]:
# =========================================================
# orders meta-data
# =========================================================

orders_meta = pd.read_csv(process_dir + "orders-split-v1.csv")
# This dataset includes the meta data for each order, i.e., the user who ordered it, order day of the week, order time
print(orders_meta.shape)

(3346083, 7)


In [10]:
display(orders_meta.head(10))

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval
0,2539329,1,1,2,8,NaN,prior
1,2398795,1,2,3,7,15.0,prior
2,473747,1,3,3,12,21.0,prior
3,2254736,1,4,4,7,29.0,prior
4,431534,1,5,4,15,28.0,prior
5,3367565,1,6,2,7,19.0,prior
6,550135,1,7,1,9,20.0,prior
7,3108588,1,8,1,14,14.0,prior
8,2295261,1,9,1,16,0.0,prior
9,2550362,1,10,4,8,30.0,prior


----

## 1.2. Merge train-val-test split

In [11]:
# =========================================================
# merge orders-wide and orders-meta
# =========================================================

orders_wide.drop(labels = "eval", axis = 1, inplace = True)
orders_wide = pd.merge(orders_wide, # data - 1
                       orders_meta[["order_id", "user_id", "eval"]], # data - 2
                       on = "order_id", # merge key
                       how = "left") # left join
print(orders_wide.shape)

(3152555, 5)


In [12]:
display(orders_wide.head(10))

,order_id,product_id,num_products,user_id,eval
0,2,33120 28985 9327 45918 30035 17794 40141 1819,8,202279,prior
1,3,33754 24838 17704 21903 17668 46667 17461 32665,8,205970,prior
2,4,46842 39758 27761 10054 21351 22598 34862 40285 17616 25146 32645 41276,12,178520,prior
3,5,13176 47329 27966 23909 48370 13245 27360 6348 40878 6184 48002 20914 37011 12962 45698 41176 48366 47209 46522 38693 48825 8479,22,156122,prior
4,7,34050 46802,2,142903,prior
5,9,21405 47890 11182 2014 29193 34203 14992 31506 23288 44533 18362 432 3990 14183,14,139016,prior
6,10,24852 4796 31717 47766 4605 1529 21137 22122 34134 27156 14992 49235 26842 3464 25720,15,135442,prior
7,11,30162 5994 1313 31506,4,143742,prior
8,12,30597 15221 43772 37886 37215 34335 26910 38888 38050 29471,10,152610,prior
9,13,17330 27407 35419 196 44635 26878 25783 41290 33198 23020 36086 3800 25952,13,45082,prior


In [13]:
# =========================================================
# eval-set distribution
# =========================================================

orders_wide["eval"].value_counts()

prior    2959079
train    116333 
val      38701  
test     38442  
Name: eval, dtype: int64

In [14]:
# clear some space
del orders_meta

----

## 1.3. Import product info data

In [15]:
products = pd.read_csv(process_dir + "products-merged-v1.csv")
print(products.shape)

(49688, 6)


In [16]:
display(products.head(10))

,product_id,product_name,aisle_id,department_id,aisle,department
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks
1,2,All-Seasons Salt,104,13,spices seasonings,pantry
2,3,Robust Golden Unsweetened Oolong Tea,94,7,tea,beverages
3,4,Smart Ones Classic Favorites Mini Rigatoni With Vodka Cream Sauce,38,1,frozen meals,frozen
4,5,Green Chile Anytime Sauce,5,13,marinades meat preparation,pantry
5,6,Dry Nose Oil,11,11,cold flu allergy,personal care
6,7,Pure Coconut Water With Orange,98,7,juice nectars,beverages
7,8,Cut Russet Potatoes Steam N' Mash,116,1,frozen produce,frozen
8,9,Light Strawberry Blueberry Yogurt,120,16,yogurt,dairy eggs
9,10,Sparkling Orange Juice & Prickly Pear Beverage,115,7,water seltzer sparkling water,beverages


In [17]:
#===============================================
# top departments
#===============================================
products["department"].value_counts()

personal care      6563
snacks             6264
pantry             5371
beverages          4365
frozen             4007
dairy eggs         3449
household          3085
canned goods       2092
dry goods pasta    1858
produce            1684
bakery             1516
deli               1322
missing            1258
international      1139
breakfast          1115
babies             1081
alcohol            1054
pets               972 
meat seafood       907 
other              548 
bulk               38  
Name: department, dtype: int64

-----

### In-class exercise

In [20]:
#===============================================
# top aisles
#===============================================
# can you figure out which aisles host the most number of products?
products["aisle"].value_counts().head(6)

missing                 1258
candy chocolate         1246
ice cream ice           1091
vitamins supplements    1038
yogurt                  1026
chips pretzels          989 
Name: aisle, dtype: int64

---

# 2. Data exploration

In [21]:
# long-form of orders
orders_long = pd.read_csv(process_dir + "all-orders-long-v1.csv")
print(orders_long.shape)

(32019330, 5)


In [22]:
# top-20 observations in the data frame
display(orders_long.head(20))

,order_id,product_id,add_to_cart_order,reordered,eval
0,2,33120,1,1,prior
1,2,28985,2,1,prior
2,2,9327,3,0,prior
3,2,45918,4,1,prior
4,2,30035,5,0,prior
5,2,17794,6,1,prior
6,2,40141,7,1,prior
7,2,1819,8,1,prior
8,3,33754,1,1,prior
9,3,24838,2,1,prior


## 2.1. Most frequently purchased products

In [23]:
# most frequently bought products
most_freq_purchased = pd.DataFrame(orders_long["product_id"].value_counts()) # count the number times each product-id appears in data frame
most_freq_purchased.reset_index(drop = False, inplace = True) # complying with pandas indexing 
most_freq_purchased.columns = ["product_id", "freq"] # assign column names
display(most_freq_purchased.head(10))

,product_id,freq
0,24852,491291
1,13176,394930
2,21137,275577
3,21903,251705
4,47209,220877
5,47766,184224
6,47626,160792
7,16797,149445
8,26209,146660
9,27845,142813


In [24]:
# merge with product info from the meta dataset
most_freq_purchased = pd.merge(most_freq_purchased, 
                               products, 
                               on = "product_id", 
                               how = "left")
display(most_freq_purchased.head(20))

,product_id,freq,product_name,aisle_id,department_id,aisle,department
0,24852,491291,Banana,24,4,fresh fruits,produce
1,13176,394930,Bag of Organic Bananas,24,4,fresh fruits,produce
2,21137,275577,Organic Strawberries,24,4,fresh fruits,produce
3,21903,251705,Organic Baby Spinach,123,4,packaged vegetables fruits,produce
4,47209,220877,Organic Hass Avocado,24,4,fresh fruits,produce
5,47766,184224,Organic Avocado,24,4,fresh fruits,produce
6,47626,160792,Large Lemon,24,4,fresh fruits,produce
7,16797,149445,Strawberries,24,4,fresh fruits,produce
8,26209,146660,Limes,24,4,fresh fruits,produce
9,27845,142813,Organic Whole Milk,84,16,milk,dairy eggs


----

### In-class exercise

In [25]:
# Can you figure out the least popular products?
display(most_freq_purchased.tail(20))

,product_id,freq,product_name,aisle_id,department_id,aisle,department
14320,15612,206,Unwrapped Bites,45,19,candy chocolate,snacks
14321,20434,206,Sardines in Olive Oil,95,15,canned meat seafood,canned goods
14322,22315,206,Slow Churned Chocolate Light Ice Cream,37,1,ice cream ice,frozen
14323,39328,206,Organic Mexican Chocolate Ice Cream,37,1,ice cream ice,frozen
14324,12549,206,"Salad Dressing, Bleu Cheese, Vegan",89,13,salad dressing toppings,pantry
14325,35355,205,94% Fat Free Microwave Butter Popcorn,23,19,popcorn jerky,snacks
14326,5634,205,Organic Pasture Raised Local Eggs,86,16,eggs,dairy eggs
14327,18975,205,Uncured Cherry Smoked Bacon,106,12,hot dogs bacon sausage,meat seafood
14328,20814,205,Sensitive with Iron Infant Formula,92,18,baby food formula,babies
14329,46815,205,Ultimate Omega 1280mg omega-3,47,11,vitamins supplements,personal care


In [26]:
# most popular aisle
aisle = most_freq_purchased.groupby("aisle").agg({"freq": "sum"})
aisle = aisle.reset_index(drop = False, inplace = False).sort_values("freq", ascending = False)
display(aisle)

,aisle,freq
50,fresh fruits,3782826
53,fresh vegetables,3553596
98,packaged vegetables fruits,1823815
133,yogurt,1475395
93,packaged cheese,990483
83,milk,918611
131,water seltzer sparkling water,868265
25,chips pretzels,717827
119,soy lactosefree,655592
11,bread,588261


----

### In-class exercise

In [29]:
# Can you figure out the most popular department, i.e., the department from where most products are purchased?
dpt = most_freq_purchased.groupby("department").agg({"freq": "sum"})
dpt = dpt.reset_index(drop = False, inplace = False).sort_values("freq", ascending = False)
display(dpt)

,department,freq
19,produce,9840710
7,dairy eggs,5522579
20,snacks,2767610
3,beverages,2641843
10,frozen,2178717
16,pantry,1763171
2,bakery,1168021
8,deli,1046768
6,canned goods,1032645
9,dry goods pasta,833181


In [96]:
# most purchased department

-----

## 2.2. Co-purchased products

In [30]:
# ===========================================
# Generate co-purchase matrix
# ===========================================

def CoPurchaseMatrix(orders_wide, product_info = True):
    '''
    orders_wide is order-level data with one row per order
    '''
    count_vec = CountVectorizer(ngram_range = (1,1), binary = True, 
                            token_pattern = "\\b\\w+\\b") # sku counts
    pur_mat = count_vec.fit_transform(orders_wide["product_id"])
    co_pur_mat = (pur_mat.T * pur_mat) #copurchase matrix
    co_pur_mat.setdiag(0) # set diagonal to 0
    co_pur_mat_df = pd.DataFrame(co_pur_mat.todense()) # convert to data frame
    co_pur_mat_df.index = count_vec.vocabulary_ # row names from sku-ids
    co_pur_mat_df.columns = count_vec.vocabulary_ # column names from sku-ids
    co_pur_mat_df = co_pur_mat_df.where(np.triu(np.ones(co_pur_mat_df.shape)).astype(np.bool)) # consider the uppre tri
    co_pur_mat_df = co_pur_mat_df.stack().reset_index() # melt to sku-1 and sku-2 per row
    co_pur_mat_df.columns = ["product_id_1", "product_id_2", "copur"]
    co_pur_mat_df = co_pur_mat_df.loc[co_pur_mat_df["copur"] > 0, :] # subset for copur > 0
    co_pur_mat_df_top = co_pur_mat_df.sort_values(["product_id_1", "copur"], ascending = False)
    co_pur_mat_df_top = co_pur_mat_df_top.drop_duplicates(["product_id_1"], keep = "first")
    co_pur_mat_df_top["product_id_1"] = co_pur_mat_df_top["product_id_1"].astype(int) # fix data types
    co_pur_mat_df_top["product_id_2"] = co_pur_mat_df_top["product_id_2"].astype(int) # fix data types
    co_pur_mat_df_top = co_pur_mat_df_top.sort_values(["copur"], ascending = False).reset_index(drop = True)
    if product_info:
        co_pur_mat_df_top = pd.merge(co_pur_mat_df_top, products, how = "left", 
                                     left_on = "product_id_1", right_on = "product_id")
        co_pur_mat_df_top.drop("product_id", axis = 1, inplace = True)
        co_pur_mat_df_top = pd.merge(co_pur_mat_df_top, products, how = "left",
                                     left_on = "product_id_2", right_on = "product_id", 
                                     suffixes = ["_1", "_2"])
        co_pur_mat_df_top.drop("product_id", axis = 1, inplace = True)
        col_order = ['product_id_1', 'product_id_2', 'copur', 'product_name_1', 'product_name_2',
                             'aisle_1', 'aisle_2', 'department_1', 'department_2', 
                             'aisle_id_1', 'aisle_id_2', 'department_id_1', 'department_id_2']
        co_pur_mat_df_top = co_pur_mat_df_top[col_order]
    return(co_pur_mat_df_top)

In [31]:
# calculate copurchases
copur = CoPurchaseMatrix(orders_wide = orders_wide, product_info = True)
print(copur.shape)

(14332, 13)


In [99]:
display(copur[["product_name_1", "product_name_2", "copur", "aisle_1", "aisle_2"]].head(20))

,product_name_1,product_name_2,copur,aisle_1,aisle_2
0,Organic Lacinato (Dinosaur) Kale,Soft Eating Strawberry Flavored Licorice,64761.0,fresh vegetables,candy chocolate
1,"Mighty 4 Sweet Potato, Blueberry, Millet & Greek Yogurt Tots Snack",Nut Delight Fruit & Nut Bar,58330.0,baby food formula,energy granola bars
2,Nut Delight Fruit & Nut Bar,Carrot Bunch,55611.0,energy granola bars,fresh vegetables
3,Natural Finely Shredded Triple Cheddar Cheese,Nut Delight Fruit & Nut Bar,53395.0,packaged cheese,energy granola bars
4,Italian Style Meatballs & Mozzarella Sandwiches,Nut Delight Fruit & Nut Bar,43180.0,frozen meals,energy granola bars
5,Organic Granny Smith Apples,Garlic Spice Blend Paste,28998.0,fresh fruits,packaged vegetables fruits
6,Pecan Pie Fruit & Nut Food Bar,Soft Eating Strawberry Flavored Licorice,26812.0,energy granola bars,candy chocolate
7,Garlic Spice Blend Paste,Carrot Bunch,25766.0,packaged vegetables fruits,fresh vegetables
8,"Almond Coconut Bar, Organic",2nd Foods Organic Pear and Spinach Baby Food,22923.0,energy granola bars,baby food formula
9,Soft Eating Strawberry Flavored Licorice,Spicy Minis Guacamole,22027.0,candy chocolate,fresh dips tapenades


----

# 3. Split train-val-test datasets


In [32]:
#===============================================
# split train-val-test
#===============================================

train = orders_wide.loc[orders_wide["eval"].isin(["prior", "train"]), :]
val = orders_wide.loc[orders_wide["eval"] == "val", :]
test = orders_wide.loc[orders_wide["eval"] == "test", :]
print("train size:", train.shape)
print("val size:", val.shape)
print("test size:", test.shape)

train size: (3075412, 5)
val size: (38701, 5)
test size: (38442, 5)


In [38]:
# clear more space
del orders_wide, orders_long

In [39]:
display(train.head())

,order_id,product_id,num_products,user_id,eval
0,2,33120 28985 9327 45918 30035 17794 40141 1819,8,202279,prior
1,3,33754 24838 17704 21903 17668 46667 17461 32665,8,205970,prior
2,4,46842 39758 27761 10054 21351 22598 34862 40285 17616 25146 32645 41276,12,178520,prior
3,5,13176 47329 27966 23909 48370 13245 27360 6348 40878 6184 48002 20914 37011 12962 45698 41176 48366 47209 46522 38693 48825 8479,22,156122,prior
4,7,34050 46802,2,142903,prior


## 3.1. Random sample for faster processing

In [40]:
#===============================================
# randomly sample training data
#===============================================

sample_size = 1000000
train = train.sample(n = sample_size)
train = train.reset_index(drop = True)
print(train.shape)

(1000000, 5)


In [41]:
display(train.head(10))

,order_id,product_id,num_products,user_id,eval
0,2572955,19836 5069 27104 42356 44234 26709 30257 28433 3801 24852 31395 45646 34050,13,178767,prior
1,924677,4920 46650 27307 24852 24841 8518 34969,7,34856,prior
2,132114,45541 38679 27845 36011 24852 27966 21137 25832,8,130089,prior
3,2579730,24852 28204 41950 37107 6046 49683 330 41596 24184 20114 26800 34450 3464 47008 22260 44303 248 16349 27521 25659 21903 38293 27104,23,96344,prior
4,1489300,43122 22825 37646 24954 33754 14437,6,154385,prior
5,2046312,38003 21333 33846 28986 46842 18418 16818 2151 24571,9,55063,prior
6,901490,24838 42768 27521 47626 22935 47209 8277 24184 46802 17948 40377 3142 4781 25126 46064,15,147251,prior
7,134163,10504 16797 40042 10960 24787 5373 25647,7,136001,prior
8,1043054,27845 40174 31683 12572 12258 33000 12206 41198 37317 33198 6182,11,39685,prior
9,1673393,47862 24964 49156,3,202319,prior


## 3.2. Tokenize sentences

In [42]:
# start pool process for parallel procressing
pool = multiprocessing.Pool(processes = cpus)

In [45]:
# text processing
from nltk import sent_tokenize, word_tokenize

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [46]:
# training sentences
t0 = time.time()
train_orders = pool.map(word_tokenize, train["product_id"])
t1 = time.time()
print(len(train_orders))

1000000


In [47]:
# time taken
print(f"Time Taken: {t1 - t0}")

Time Taken: 63.63620114326477


In [48]:
# training orders
train_orders[0:3]

[['19836',
  '5069',
  '27104',
  '42356',
  '44234',
  '26709',
  '30257',
  '28433',
  '3801',
  '24852',
  '31395',
  '45646',
  '34050'],
 ['4920', '46650', '27307', '24852', '24841', '8518', '34969'],
 ['45541', '38679', '27845', '36011', '24852', '27966', '21137', '25832']]

In [49]:
# validation sentences
t0 = time.time()
val_orders = pool.map(word_tokenize, val["product_id"])
t1 = time.time()
print(len(val_orders))

38701


----

### In-class exercise

In [111]:
# what do the first three validation orders look like?

In [112]:
# Can you similarly tokenize the test sentences?
#print(len(test_orders))

----

# 4. Word2Vec sample model

## 4.1. Define and train model

In [60]:
print(type(train_orders))
print(len(train_orders))

<class 'list'>
1000000


In [50]:
#===============================================
# define and train model
#===============================================

w2v_1 = gensim.models.Word2Vec(sentences = train_orders,
                               workers = multiprocessing.cpu_count(),
                               seed = 1234)

## Model properties

In [51]:
#===============================================
# vocabulary
#===============================================

# vocabulary length
f"Vocab length: {len(w2v_1.wv.vocab)}"

'Vocab length: 14340'

In [52]:
# sample vocabulary
list(w2v_1.wv.vocab.keys())[0:5]

['19836', '5069', '27104', '42356', '44234']

In [53]:
# length of corpus
print(w2v_1.corpus_count)

1000000


In [54]:
# number of iterations
w2v_1.iter

5

In [55]:
#===============================================
# product vectors
#===============================================

# enter product-id
prod_id = "1"
print(w2v_1[prod_id].shape)
print("------------")
print("------------")
print(w2v_1[prod_id])

(100,)
------------
------------
[-4.04510736e-01  5.81333399e-01  4.53743726e-01 -3.50309789e-01
  1.01281011e+00 -4.07895654e-01 -5.56387961e-01  5.87131798e-01
  8.55092287e-01  9.39807117e-01  7.45321512e-01 -8.14930975e-01
 -8.72512102e-01  7.91543871e-02  9.06610668e-01 -3.82677168e-01
 -4.74929512e-01 -5.41775189e-02 -2.49912918e-01 -1.83316660e+00
 -8.09821561e-02  8.36983323e-01 -4.07992959e-01  1.27453935e+00
  6.72552466e-01 -1.81895447e+00 -1.16033173e+00  9.22205448e-01
 -1.01271415e+00 -4.68485922e-01 -8.93574238e-01  1.16683180e-02
 -4.44470018e-01 -4.93991613e-01  5.31884909e-01  4.11142975e-01
 -8.66689801e-01  8.19667816e-01 -8.06315720e-01 -9.77191567e-01
 -5.36102057e-01  2.46548280e-01  2.07711667e-01  4.12979960e-01
  8.24673533e-01  1.07451487e+00 -4.57732707e-01 -7.52725959e-01
 -4.10204858e-01 -8.96955729e-02  3.86765540e-01 -3.32888126e-01
 -4.38780040e-01  1.95192918e-01 -9.68766034e-01  1.15196693e+00
  3.79069000e-01 -5.19667804e-01  8.28152597e-01  1.85116

## 4.2. Update embeddings

In [56]:
#===============================================
# update model weights
#===============================================
w2v_1.train(sentences = train_orders, total_examples = w2v_1.corpus_count, epochs = 2)

(19314833, 20196396)

## 4.3. Inspect model output

In [57]:
#===============================================
# upadted product vectors
#===============================================

# enter product-id
prod_id = "1"
print(w2v_1[prod_id].shape)
print("------------")
print("------------")
print(w2v_1[prod_id])

(100,)
------------
------------
[-0.7753562   1.101924    0.79193944 -0.1209721   0.9255502  -0.49209473
 -0.5375001   0.66311926  1.2073712   1.0305341   0.8318689  -0.92228687
 -1.0315843  -0.13880645  0.74502397 -0.19811006 -0.31725487 -0.27960244
 -0.27866358 -1.96529     0.17912069  0.7192371  -0.22476996  1.4866871
  0.8997672  -2.232609   -1.09567     1.2788435  -0.9064601  -0.9304165
 -0.79422253 -0.09606735 -0.47052908 -0.4087471   0.67744124  0.01524378
 -1.0514189   1.0494012  -0.72372305 -1.2782072  -0.4718718   0.09444378
  0.45311087  0.68848443  0.7339318   1.3622518  -0.67777956 -0.6778548
 -0.52416694 -0.17035656  0.34380886 -0.34908834 -0.53268635  0.17979494
 -0.9853808   1.1113174   0.6251342  -0.37770325  0.66359603  1.9883057
  0.06227816 -1.2932522  -0.1873026  -0.52454215  0.5209079   0.1737606
 -0.0990522  -0.10883834  0.18628901 -0.3774401   0.32058585 -0.06658533
  0.03432867  0.40749118 -1.217024   -0.6472818   0.39767978  0.27677667
  0.4414988  -0.6221793

--------

# 5. Improve W2V model

In [61]:
#===============================================
# model parameters
#===============================================

# size of embedding matrix
emb_size = 100

# context window size
cxt_window = 10

# batch size for gradient update
batch_size = 10000

# down-sample high frequency words
hfs = 0.001

# learning rate
lr = 0.05

In [62]:
#===============================================
# define model
#===============================================

w2v_1 = gensim.models.Word2Vec(sentences = train_orders,
                               size = emb_size, # number of columns in embedding matrix
                               hs = 1, # hierarchical softmax
                               negative = 0, # negative sampling
                               window = cxt_window, # context window
                               min_count = 1, # minimum frequency count
                               batch_words = batch_size, # batch size for update
                               alpha = lr, # learning rate
                               sample = hfs, # down sample high frequency words 
                               workers = cpus,
                               seed = 1234)

## 5.1. Score on validation and test

In [69]:
#===============================================
# Score model to get log-likelihood
#===============================================

def ScoreW2V(test_sent, model, normalize = True, avg_over_sent = True):
    test_score = model.score(test_sent, total_sentences = len(test_sent), 
                             chunksize = 100,
                             queue_factor = 2,
                             report_delay = 1)
    if normalize:
        test_score = [test_score[x]/len(test_sent[x]) for x in range(len(test_sent))]
    else:
        test_score = list(test_score)
    if avg_over_sent:
        test_score = np.mean(test_score)
    return test_score

In [64]:
w2v_1_val = ScoreW2V(test_sent = val_orders, model = w2v_1, normalize = True, avg_over_sent = True)

In [65]:
w2v_1_val

-7.527941547609724

## 5.2. Similar products

In [66]:
#===============================================
# similar products
#===============================================

# sample product
prod_id = "10"

# product info for sample product
display(products.loc[products["product_id"].isin([prod_id]), :])

,product_id,product_name,aisle_id,department_id,aisle,department


In [67]:
#===============================================
# similarity from model
#===============================================

w2v_1.wv.most_similar(prod_id)

[('44375', 0.7991302609443665),
 ('6475', 0.6662318110466003),
 ('4138', 0.6114770174026489),
 ('4493', 0.5634533762931824),
 ('17080', 0.5379359126091003),
 ('8192', 0.4694351255893707),
 ('32380', 0.4646366238594055),
 ('28004', 0.44108134508132935),
 ('19125', 0.42980924248695374),
 ('21006', 0.413822740316391)]

In [128]:
#===============================================
# lookup product info
#===============================================
most_similar_prods = [y[0] for y in w2v_1.wv.most_similar(positive = prod_id)]
most_similar_prods = products.loc[products["product_id"].isin(most_similar_prods), :]
display(most_similar_prods)

,product_id,product_name,aisle_id,department_id,aisle,department
455,456,Sparkling Blueberry,77,7,soft drinks,beverages
4137,4138,Arancita Rossa,77,7,soft drinks,beverages
4492,4493,Italian Sparkling Pomegranate And Orange Soda,115,7,water seltzer sparkling water,beverages
5882,5883,Organic Strawberry Lemonade,98,7,juice nectars,beverages
6474,6475,Limonata Sparkling Lemon Beverage,77,7,soft drinks,beverages
14765,14766,Sparkling Grapefruit,77,7,soft drinks,beverages
17079,17080,Clementina Sparkling Beverage,115,7,water seltzer sparkling water,beverages
18388,18389,Blood Orange Italian Soda,77,7,soft drinks,beverages
28003,28004,Limonata Sparkling Beverage,115,7,water seltzer sparkling water,beverages
44374,44375,Canned Aranciata Orange,77,7,soft drinks,beverages


In [129]:
#===============================================
# most dissimilar products
#===============================================
most_dissimilar_prods = [y[0] for y in w2v_1.wv.most_similar(negative = [prod_id])]
most_dissimilar_prods = products.loc[products["product_id"].isin(most_dissimilar_prods), :]
display(most_dissimilar_prods)

,product_id,product_name,aisle_id,department_id,aisle,department
951,952,"Flaxseed Meal, Whole Ground",17,13,baking ingredients,pantry
1246,1247,Sloppy Joes Seasoning Mix,5,13,marinades meat preparation,pantry
17510,17511,Crunchy Seven Nut & Seed Butter,88,13,spreads,pantry
17595,17596,Fruity Cheerios Cereal,121,14,cereal,breakfast
18170,18171,Natural Sunflower Spread,88,13,spreads,pantry
22039,22040,Fresh Cut Blue Lake No Salt Added Cut Green Beans,81,15,canned jarred vegetables,canned goods
34853,34854,"Steamfresh Selects Frozen Broccoli, Cauliflower & Carrots",116,1,frozen produce,frozen
40375,40376,Steamfresh Sweet Peas,116,1,frozen produce,frozen
40620,40621,Broccoli & Cauliflower,116,1,frozen produce,frozen
45964,45965,Steel Cut Oats,130,14,hot cereal pancake mixes,breakfast


----

### In-class exercise

In [130]:
# find the most similar products to 
prod_id = "100"

# product info for sample product
display(products.loc[products["product_id"].isin([prod_id]), :])

,product_id,product_name,aisle_id,department_id,aisle,department
99,100,Peanut Butter & Strawberry Jam Sandwich,38,1,frozen meals,frozen


In [131]:
# which products are most similar to peanut better and strawberry jam sandwich

In [132]:
# what about the most dissimilar products?

----

# 5. Product maps using t-sne 

## 5.1. Extract all product vectors

In [133]:
#===============================================
# product vectors
#===============================================

items = [key for key in w2v_1.wv.vocab.keys()] # all product-ids
word_vec = [list(w2v_1[item]) for item in items] # "word" vector for each product id
word_vec = np.array(word_vec) # convert to array
word_vec_df = pd.DataFrame(word_vec) # convert to data frame
w2v_vec_names = ["wv" + str(x + 1) for x in range(word_vec_df.shape[1])] # column names
word_vec_df.columns = w2v_vec_names # assign column names
word_vec_df["product_id"] = items # include product id in data frame
word_vec_df["product_id"] = word_vec_df["product_id"].astype(int) # convert to type integer for later merge
word_vec_df = word_vec_df[["product_id"] + w2v_vec_names] # re-order columns
print(word_vec_df.shape)

(14340, 101)


In [134]:
display(word_vec_df.head(10))

,product_id,wv1,wv2,wv3,wv4,wv5,wv6,wv7,wv8,wv9,wv10,wv11,wv12,wv13,wv14,wv15,wv16,wv17,wv18,wv19,wv20,wv21,wv22,wv23,wv24,wv25,wv26,wv27,wv28,wv29,wv30,wv31,wv32,wv33,wv34,wv35,wv36,wv37,wv38,wv39,wv40,wv41,wv42,wv43,wv44,wv45,wv46,wv47,wv48,wv49,wv50,wv51,wv52,wv53,wv54,wv55,wv56,wv57,wv58,wv59,wv60,wv61,wv62,wv63,wv64,wv65,wv66,wv67,wv68,wv69,wv70,wv71,wv72,wv73,wv74,wv75,wv76,wv77,wv78,wv79,wv80,wv81,wv82,wv83,wv84,wv85,wv86,wv87,wv88,wv89,wv90,wv91,wv92,wv93,wv94,wv95,wv96,wv97,wv98,wv99,wv100
0,27888,0.991016,-0.055347,-0.618443,-0.112379,0.845730,-0.161779,-0.235410,-0.780843,-0.411044,1.131562,0.257362,-0.023040,0.325801,-1.278199,0.935081,0.000162,0.693156,1.378203,-0.097100,1.450118,-0.849971,0.208069,0.172751,0.453095,-0.224899,0.798142,-0.861846,0.218438,-2.469534,-1.191987,0.312043,1.023138,-0.282084,-0.693851,0.873696,-0.546057,-1.030167,-0.544695,-0.800045,0.457829,0.370236,0.657401,-0.654934,-0.689802,-0.332016,-0.374241,0.932325,-1.502272,-0.258942,0.085221,-0.035942,-0.226732,0.716972,0.753459,0.702796,-0.443695,-0.010270,-0.230515,0.254941,0.204216,0.535912,-0.457416,-0.701698,-1.841580,0.732887,-0.750403,-0.144997,0.118411,-0.243787,-1.045582,1.049450,-0.562250,0.684943,0.306815,0.709360,-0.453412,0.323479,-1.063863,0.113524,-1.039436,-0.081299,1.041267,-0.461687,-0.756298,-0.168442,0.976666,-0.112870,0.614994,-0.269176,-0.582358,0.928558,-1.494900,-0.043209,0.213914,1.543325,-0.165199,-1.531970,-1.706078,0.239023,-0.962737
1,21137,0.488940,-1.071994,-0.169053,0.005896,0.645513,0.226873,0.008350,-0.393018,-0.112132,0.418457,0.209269,0.490734,0.599445,0.400796,-0.235424,0.461379,-0.198410,-0.260389,-0.210395,0.083492,-0.417034,0.250042,-0.369893,0.436977,0.134372,0.377746,-1.016585,0.177668,0.142176,-0.281023,0.159387,0.085806,0.428145,-0.171543,0.399599,0.580232,0.023975,-0.551147,0.521113,0.201741,0.457943,0.602929,-0.450980,-0.309515,-0.022506,0.562622,-0.366060,-0.868004,-0.482000,0.020069,0.228861,-0.100942,-0.268868,0.260370,-0.066975,-0.480731,-0.036262,0.618339,-0.224918,0.118141,0.027317,-0.581062,-0.175178,-0.592972,-0.452428,-0.660195,-0.810445,-0.743833,0.263803,0.820481,-0.176059,-0.167931,0.008489,0.064380,-0.284380,0.423906,0.074715,-0.066025,-0.053279,-0.015526,0.540443,0.221875,0.008151,-0.161192,-0.361980,-0.481994,-0.482474,0.057734,-0.114625,-0.342058,-0.060914,0.561120,0.252346,0.225239,0.093766,-0.403118,-0.465855,0.036376,0.202066,-0.187435
2,46979,0.585247,-0.580192,0.229223,-0.557701,-0.212620,-0.392506,-0.768170,-0.566538,-0.408713,-0.566110,0.284612,0.372882,-0.368035,0.246818,0.441033,0.187344,0.093674,0.079909,0.107954,0.007604,-0.041029,0.614836,-0.199368,0.357653,0.013743,-0.256645,0.367435,-0.270438,0.148032,-0.074880,-0.037992,-0.251790,0.129986,0.213501,0.704362,-0.478214,-0.416858,0.273656,-0.275017,-0.998560,-0.591249,0.595047,0.185176,-0.174462,-0.069881,-0.122427,-0.934116,0.034601,-0.609276,0.132374,-0.003258,0.282898,-0.673007,-0.262482,-0.460662,-0.609691,0.023730,0.180362,-0.152889,0.327437,0.308289,0.127156,0.671862,1.021824,-0.541045,0.595015,-0.235154,0.191937,0.248536,0.452162,-0.278580,0.155287,0.165723,0.139932,0.121873,0.442441,-0.505026,-0.658693,0.373433,-0.450443,0.104672,0.409075,0.586718,0.233794,-0.282450,0.235543,-0.059182,-0.401147,0.046142,-0.032179,0.224874,-0.054540,-0.246128,-0.104254,-0.178707,0.250769,-0.332059,-0.043698,-0.269527,-0.413866
3,21903,0.434819,-0.224153,0.012649,-0.553659,-0.075412,-0.257416,-0.385184,-0.547945,0.124002,0.070073,0.032035,-0.001860,-0.308162,-0.003908,-0.380619,0.680212,-0.089089,-0.058465,0.005208,-0.445409,-0.342089,0.757735,-0.251449,0.003823,-0.086070,0.441477,-0.338572,-0.219321,0.281067,-0.295155,-0.480073,-0.257283,0.112104,0.463972,0.371576,0.188066,0.082587,0.278639,-0.302790,-0.650368,-0.770150,0.428568,-0.357414,-0.210241,0.194125,-0.463022,-0.677743,-0.206159,-0.182604,0.163094,0.043676,-0.274664,-0.396438,0.023866,-0.103760,-0.447145,-0.123595,0.730513,0.029890,0.597319,0.066663,0.476858,0.098387,-0.0634

## Merge product vectors with product info

In [135]:
#===============================================
# merge relevant columns
#===============================================

# relevant columns from product info
prod_info_cols = ["product_id", "product_name", "department", "aisle"]

# merge
word_vec_df = pd.merge(products[prod_info_cols], word_vec_df, on = "product_id", how = "inner")
print(word_vec_df.shape)

(14340, 104)


In [136]:
display(word_vec_df.head(10))

,product_id,product_name,department,aisle,wv1,wv2,wv3,wv4,wv5,wv6,wv7,wv8,wv9,wv10,wv11,wv12,wv13,wv14,wv15,wv16,wv17,wv18,wv19,wv20,wv21,wv22,wv23,wv24,wv25,wv26,wv27,wv28,wv29,wv30,wv31,wv32,wv33,wv34,wv35,wv36,wv37,wv38,wv39,wv40,wv41,wv42,wv43,wv44,wv45,wv46,wv47,wv48,wv49,wv50,wv51,wv52,wv53,wv54,wv55,wv56,wv57,wv58,wv59,wv60,wv61,wv62,wv63,wv64,wv65,wv66,wv67,wv68,wv69,wv70,wv71,wv72,wv73,wv74,wv75,wv76,wv77,wv78,wv79,wv80,wv81,wv82,wv83,wv84,wv85,wv86,wv87,wv88,wv89,wv90,wv91,wv92,wv93,wv94,wv95,wv96,wv97,wv98,wv99,wv100
0,1,Chocolate Sandwich Cookies,snacks,cookies cakes,0.744708,1.295512,-1.193265,0.598794,-0.641248,1.223336,0.919235,0.809076,0.790224,-1.406452,0.579844,-0.417504,0.320172,-1.006021,-1.247783,0.728549,0.282868,-0.163652,0.442921,0.128002,1.018741,-1.040465,0.126721,0.412622,0.333163,0.891142,-0.438378,-0.414526,-0.543069,-1.434892,-0.369650,-0.234173,-0.450169,0.144263,-0.889828,-0.092579,0.095353,-0.893399,-0.008923,-0.703940,-0.921108,-1.429784,0.343726,-0.504728,-1.583240,0.781951,-0.361618,-0.487692,0.108233,-1.003854,0.445510,0.412661,2.036789,0.664207,-0.936389,0.153528,-0.561427,0.708672,-1.219241,-0.778229,0.468867,-0.237254,-0.688795,0.463018,0.221844,-1.234477,-0.520427,0.230692,-1.529698,0.867693,-0.802470,-0.222265,-0.507642,0.082833,-0.856651,0.124394,-0.133245,-1.038114,-0.809275,-0.205799,-0.915241,-0.091222,-0.302496,0.530782,0.226141,-0.134771,-0.599206,-0.370918,0.447913,0.089710,-0.735691,-0.266758,-1.274789,-1.471167,0.414221,-0.482969,-0.722963,-0.789088,-0.890457,0.141746
1,3,Robust Golden Unsweetened Oolong Tea,beverages,tea,-0.156999,-0.097948,1.033049,-0.075712,-0.156379,0.859535,-0.202249,-0.692840,1.155985,-0.154656,1.157791,-0.287568,0.291294,0.716055,-0.463947,-0.266470,-0.432250,-0.462481,-0.609656,0.939700,0.484884,-0.209098,0.338069,1.126593,-0.584675,-1.762120,-1.089806,1.540631,0.531554,0.231101,0.813328,0.308757,-1.116319,0.098051,0.718829,0.123319,-0.361023,-0.134135,0.584352,-0.445583,0.308385,0.375197,-0.108583,-1.362428,0.132718,-1.448387,-0.245276,1.475555,1.410261,-0.275512,0.245381,-0.178379,0.160996,-0.317731,-0.286790,-0.621966,1.150026,-1.059867,-1.085093,-0.115807,0.323545,-1.090988,0.613781,0.329271,1.433473,0.560549,-0.153129,-0.580917,-1.053341,0.052376,-0.343750,1.419617,0.731421,0.625508,0.882422,-0.762834,-0.411799,0.003922,-1.166005,-0.027614,0.316082,-0.358464,-0.382854,1.930140,-0.251959,0.190719,0.468010,0.382750,0.392147,-0.921297,0.853766,0.890511,0.992793,0.780575,0.511955,0.541127,-0.399190,0.485569,-0.989308,-0.412728
2,4,Smart Ones Classic Favorites Mini Rigatoni With Vodka Cream Sauce,frozen,frozen meals,0.632169,-0.042714,0.447641,0.565454,-0.173308,-1.259952,-0.374098,0.565182,-0.501129,-0.326884,-1.161234,0.588260,1.618659,0.294504,0.154790,0.866748,-1.177649,-0.134442,-0.567716,-0.921026,0.309276,-0.598171,2.971751,-0.787572,-0.649750,-0.231039,1.806901,1.559718,0.501790,0.281565,0.014173,0.633412,0.325695,0.359038,-0.070100,0.124627,-0.962404,-0.134815,-0.145717,1.183958,0.241399,0.964502,1.174005,1.029604,0.155260,-0.624118,-0.044555,0.265823,1.074033,-1.058899,-0.206956,0.068395,1.224924,0.802075,-0.673816,2.257773,-1.068509,0.063531,1.522198,0.739853,1.355033,0.423484,0.501739,0.359911,0.319578,-0.454862,0.050564,0.328704,0.205272,-0.856989,0.670041,1.876261,1.597733,1.334862,-0.317515,-0.957292,-1.040504,-0.442202,-0.018599,0.246100,0.799682,-0.482413,-1.251701,-1.626282,-0.359774,0.296589,-0.049335,0.718455,1.660907,-0.985967,2.556258,-0.344786,-0.415751,-1.505674,-0.344438,0.000373,-0.931319,2.007142,-0.076104,-0.533512
3,10,Sparkling Orange Juice & Prickly Pear Beverage,beverages,water seltzer sparkling water,-0.253488,0.374834,-1.069590,0.956164,-0.311371,1.789404,0.267260,0.387516,-0.654127,1.027248,1.398794,0.410207,0.221405,1.188907,0.842219,0.270739,-1.056333,-1.884760,1.011292,0.223562,-0.034032,-0.055429,0.418091,0.480409,1.238376,0.885619,0.360691,3.544943,-0.051665,-0.017406,-0.987518,0.134707,-0.523529,0.258269,-0.3047

## 5.2. Fit t-sne

In [137]:
# =================================================
# fit t-sne
# =================================================
tsne = TSNE(n_components = 2, verbose = 1, perplexity = 35, n_iter = 400)

In [138]:
# fit
t0 = time.time()
tsne_fit = tsne.fit_transform(word_vec_df[w2v_vec_names])
t1 = time.time()

[t-SNE] Computing 106 nearest neighbors...
[t-SNE] Indexed 14340 samples in 0.077s...
[t-SNE] Computed neighbors for 14340 samples in 24.032s...
[t-SNE] Computed conditional probabilities for sample 1000 / 14340
[t-SNE] Computed conditional probabilities for sample 2000 / 14340
[t-SNE] Computed conditional probabilities for sample 3000 / 14340
[t-SNE] Computed conditional probabilities for sample 4000 / 14340
[t-SNE] Computed conditional probabilities for sample 5000 / 14340
[t-SNE] Computed conditional probabilities for sample 6000 / 14340
[t-SNE] Computed conditional probabilities for sample 7000 / 14340
[t-SNE] Computed conditional probabilities for sample 8000 / 14340
[t-SNE] Computed conditional probabilities for sample 9000 / 14340
[t-SNE] Computed conditional probabilities for sample 10000 / 14340
[t-SNE] Computed conditional probabilities for sample 11000 / 14340
[t-SNE] Computed conditional probabilities for sample 12000 / 14340
[t-SNE] Computed conditional probabilities for s

In [139]:
# time taken
f"Time Taken: {t1 - t0}"

'Time Taken: 187.86533784866333'

## T-sne component data frame

In [140]:
#===============================================
# create t-sne data frame
#===============================================

tsne_df = word_vec_df[["product_name", "department", "aisle"]]

# extract t-sne dimensions
tsne_df["x_tsne"] = tsne_fit[:,0]
tsne_df["y_tsne"] = tsne_fit[:,1]
print(tsne_df.describe())

             x_tsne        y_tsne
count  14340.000000  14340.000000
mean   0.081110     -0.013953    
std    10.179037     7.783475    
min   -22.075830    -19.978592   
25%   -7.855527     -5.532616    
50%   -0.820201     -0.028690    
75%    8.695611      5.579124    
max    22.841021     21.091007   


## Plot with ggplot

In [141]:
#===============================================
# subset data for plot
#===============================================

# select only top departments
select_dept = ["produce", "babies", "beverages"]
tsne_plot_df = tsne_df.loc[tsne_df["department"].isin(select_dept), :]
print(tsne_plot_df.shape)

(2726, 5)


In [ ]:
tsne_plot = ggplot(tsne_plot_df, aes(x = "x_tsne", y = "y_tsne", color = "department") ) \
        + geom_point(size = 70, alpha = 0.5) \
        + ggtitle("T-sne on product vectors") \
        + xlab(" ") + ylab(" ")
tsne_plot

---

## In-class exercise

## Learning embeddings using Skip-gram

- We will now use another method to train embeddings called skip-gram

In [ ]:
#===============================================
# model parameters
#===============================================

# size of embedding matrix
emb_size  # set between 50-100

# context window size
cxt_window # set between 2-10 

# batch size for gradient update
batch_size # set between 2000 to 10000

# learning rate
lr # set between 0.001 to 0.01

In [ ]:
#===============================================
# define and train model
#===============================================

t0 = time.time()
### Write model code here
t1 = time.time()

In [ ]:
# time taken
f"Time Taken: {t1 - t0}"

In [ ]:
# find the most similar products to 
prod_id = "1000"

# product info for sample product
display(products.loc[products["product_id"].isin([prod_id]), :])